In [1]:
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt

from imutils import paths
from tqdm import tqdm
import os
import cv2
import numpy as np


In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


# Yêu cầu 1

In [37]:
class NeuralNetwork_1(nn.Module):
    def __init__(self):
        super(NeuralNetwork_1, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=2, padding=1)

        # self.depthwise_conv1 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=57, groups=32)
        self.depthwise_conv1 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1, groups=32)  # Depthwise convolution
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')
        self.pointwise_conv1 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=1, stride=1, padding = 0)

        self.depthwise_conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=2, padding=1, groups=64)
        self.pointwise_conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=1, stride=1, padding = 0)

        self.depthwise_conv3 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1, groups=128)
        self.pointwise_conv3 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=1, stride=1, padding = 0)

        self.depthwise_conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=2, padding = 1, groups = 128)
        self.pointwise_conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=1, stride=1, padding = 0)

        self.depthwise_conv5 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, groups=256)  # Depthwise convolution
        self.pointwise_conv5 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=1, stride=1, padding = 0)

        self.depthwise_conv6 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=2, padding=1, groups=256)
        self.pointwise_conv6 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=1, stride=1, padding = 0)

        self.five_conv = nn.ModuleList([
        nn.Sequential(
                nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=2, padding=1, groups=512),  # 3x3 Depthwise Conv
                nn.Upsample(scale_factor=2),
                nn.Conv2d(in_channels=512, out_channels=512, kernel_size=1, stride=1, padding=0)  # 1x1 Pointwise Conv
            ) for _ in range(5)
        ])

        self.depthwise_conv7 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=2, padding=1, groups=512)  # Depthwise convolution
        self.pointwise_conv7 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=1, stride=1, padding = 0)

        self.depthwise_conv8 = nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, stride=1, padding=1, groups=1024)
        self.pointwise_conv8 = nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=1, stride=1, padding = 0)

        self.avg_pool = nn.AvgPool2d(kernel_size=7, stride=1)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(in_features=1024, out_features=101)




    def forward(self, x):
      x = F.relu(self.conv1(x))

      x = F.relu(self.depthwise_conv1(x))
      x = self.upsample(x)
      x = F.relu(self.pointwise_conv1(x))

      x = F.relu(self.depthwise_conv2(x))
      x = F.relu(self.pointwise_conv2(x))

      x = F.relu(self.depthwise_conv3(x))
      x = F.relu(self.pointwise_conv3(x))

      x = F.relu(self.depthwise_conv4(x))
      x = F.relu(self.pointwise_conv4(x))

      x = F.relu(self.depthwise_conv5(x))
      x = F.relu(self.pointwise_conv5(x))

      x = F.relu(self.depthwise_conv6(x))
      x = F.relu(self.pointwise_conv6(x))

      for conv in self.five_conv:
            x = F.relu(conv(x))

      x = F.relu(self.depthwise_conv7(x))
      x = F.relu(self.pointwise_conv7(x))

      x = F.relu(self.depthwise_conv8(x))
      x = F.relu(self.pointwise_conv8(x))

      x = self.avg_pool(x)
      x = self.flatten(x)
      x = self.fc(x)

      return x

In [4]:
learning_rate = 0.0001
num_epochs = 10
batch_size = 32

In [5]:
#Data Preprocesing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
# Load Caltech101 dataset
data_path='./'
dataset = datasets.Caltech101(data_path, download=True, transform=transform)

Downloading...
From (original): https://drive.google.com/uc?id=137RyRjvTBkBiIfeYBNZBtViDHQ6_Ewsp
From (redirected): https://drive.usercontent.google.com/download?id=137RyRjvTBkBiIfeYBNZBtViDHQ6_Ewsp&confirm=t&uuid=7477523f-0b34-48c9-b9e1-d38b99d2c495
To: /content/caltech101/101_ObjectCategories.tar.gz
100%|██████████| 132M/132M [00:01<00:00, 121MB/s]


Extracting ./caltech101/101_ObjectCategories.tar.gz to ./caltech101


Downloading...
From (original): https://drive.google.com/uc?id=175kQy3UsZ0wUEHZjqkUDdNVssr7bgh_m
From (redirected): https://drive.usercontent.google.com/download?id=175kQy3UsZ0wUEHZjqkUDdNVssr7bgh_m&confirm=t&uuid=f03ce377-8de0-4589-b487-4472a5622780
To: /content/caltech101/Annotations.tar
100%|██████████| 14.0M/14.0M [00:00<00:00, 64.7MB/s]


Extracting ./caltech101/Annotations.tar to ./caltech101


In [7]:
# Split the dataset into training, validation, and testing sets
train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size + test_size])
val_dataset, test_dataset = torch.utils.data.random_split(val_test_dataset, [val_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [9]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = (self.alpha * (1 - pt) ** self.gamma * ce_loss) if self.alpha is not None else (1 - pt) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return torch.mean(focal_loss)
        elif self.reduction == 'sum':
            return torch.sum(focal_loss)
        else:
            return focal_loss

In [ ]:
model = NeuralNetwork_1().to(device)

In [10]:
# Training loop
def training(opttimize = 'Adam', lossFunction = 'CrossEntropy'):
  if opttimize == 'Adam' :
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
  elif opttimize == 'SGD':
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

  if lossFunction == 'CrossEntropy':
    criterion = nn.CrossEntropyLoss()
  elif lossFunction == 'FocalLoss':
    criterion = FocalLoss().to(device)
  for epoch in range(num_epochs):
      for i, (images, labels) in enumerate(tqdm(train_loader)):
          images = images.to(device)
          labels = labels.to(device)

          # Forward pass
          outputs = model(images)
          loss = criterion(outputs, labels)

          # Backward and optimize
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          if (i+1) % 100 == 0:
              print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

  print("Training finished!")

  # Testing the model
  model.eval() # Set the model to evaluation mode
  with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f"Test accuracy of the model on the {total} test images: {correct / total}")

In [11]:
training('Adam', 'CrossEntropy')

 62%|██████▏   | 101/163 [00:12<00:08,  7.31it/s]

Epoch [1/10], Step [100/163], Loss: 4.1552


 63%|██████▎   | 102/163 [00:10<00:05, 10.57it/s]

Epoch [2/10], Step [100/163], Loss: 3.8909


 62%|██████▏   | 101/163 [00:10<00:06, 10.21it/s]

...

Epoch [9/10], Step [100/163], Loss: 4.0580


 63%|██████▎   | 102/163 [00:10<00:05, 10.39it/s]

Epoch [10/10], Step [100/163], Loss: 4.2949


100%|██████████| 163/163 [00:15<00:00, 10.44it/s]


Training finished!
Test accuracy of the model on the 1736 test images: 10.023041474654377%


In [12]:
training('Adam', 'FocalLoss')

 62%|██████▏   | 101/163 [00:11<00:06,  9.82it/s]

Epoch [1/10], Step [100/163], Loss: 4.1420


 62%|██████▏   | 101/163 [00:10<00:05, 10.36it/s]

Epoch [2/10], Step [100/163], Loss: 3.5339


 62%|██████▏   | 101/163 [00:09<00:07,  8.28it/s]

...

Epoch [9/10], Step [100/163], Loss: 4.1533


 62%|██████▏   | 101/163 [00:09<00:05, 10.41it/s]

Epoch [10/10], Step [100/163], Loss: 3.9190


100%|██████████| 163/163 [00:15<00:00, 10.32it/s]


Training finished!
Test accuracy of the model on the 1736 test images: 8.46774193548387%


In [13]:
training('SGD', 'CrossEntropy')

 63%|██████▎   | 102/163 [00:09<00:05, 10.44it/s]

Epoch [1/10], Step [100/163], Loss: 4.2066


 63%|██████▎   | 102/163 [00:09<00:06,  9.73it/s]

Epoch [2/10], Step [100/163], Loss: 4.0923


 62%|██████▏   | 101/163 [00:09<00:06, 10.27it/s]

...

Epoch [9/10], Step [100/163], Loss: 4.2366


 63%|██████▎   | 102/163 [00:10<00:06,  9.46it/s]

Epoch [10/10], Step [100/163], Loss: 4.3391


100%|██████████| 163/163 [00:15<00:00, 10.49it/s]


Training finished!
Test accuracy of the model on the 1736 test images: 8.46774193548387%


In [14]:
training('SGD', 'FocalLoss')

 63%|██████▎   | 102/163 [00:09<00:05, 10.57it/s]

Epoch [1/10], Step [100/163], Loss: 4.3488


 62%|██████▏   | 101/163 [00:09<00:08,  6.90it/s]

Epoch [2/10], Step [100/163], Loss: 3.7681


 62%|██████▏   | 101/163 [00:09<00:06, 10.32it/s]

...

Epoch [9/10], Step [100/163], Loss: 3.9802


 63%|██████▎   | 102/163 [00:08<00:05, 10.60it/s]

Epoch [10/10], Step [100/163], Loss: 4.3177


100%|██████████| 163/163 [00:15<00:00, 10.72it/s]


Training finished!
Test accuracy of the model on the 1736 test images: 8.46774193548387%


# Yêu cầu 2

In [49]:
class NeuralNetwork_2(nn.Module):
    def __init__(self):
        super(NeuralNetwork_2, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3)

        self.depthwise_conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=2, padding=1, groups=64)
        self.pointwise_conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=1, stride=1, padding = 0)

        self.depthwise_conv3 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1, groups=128)
        self.pointwise_conv3 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=1, stride=1, padding = 0)

        self.depthwise_conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=2, padding = 1, groups = 128)
        self.pointwise_conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=1, stride=1, padding = 0)

        self.depthwise_conv5 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, groups=256)  # Depthwise convolution
        self.pointwise_conv5 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=1, stride=1, padding = 0)

        self.depthwise_conv6 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=2, padding=1, groups=256)
        self.pointwise_conv6 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=1, stride=1, padding = 0)

        self.five_conv = nn.ModuleList([
        nn.Sequential(
                nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=2, padding=1, groups=512),  # 3x3 Depthwise Conv
                nn.Upsample(scale_factor=2),
                nn.Conv2d(in_channels=512, out_channels=512, kernel_size=1, stride=1, padding=0)  # 1x1 Pointwise Conv
            ) for _ in range(5)
        ])

        self.depthwise_conv7 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=2, padding=1, groups=512)  # Depthwise convolution
        self.pointwise_conv7 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=1, stride=1, padding = 0)

        self.depthwise_conv8 = nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, stride=1, padding=1, groups=1024)
        self.pointwise_conv8 = nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=1, stride=1, padding = 0)

        self.avg_pool = nn.AvgPool2d(kernel_size=7, stride=1)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(in_features=1024, out_features=101)




    def forward(self, x):
      x = F.relu(self.conv1(x))
      x = F.relu(self.depthwise_conv2(x))
      x = F.relu(self.pointwise_conv2(x))

      x = F.relu(self.depthwise_conv3(x))
      x = F.relu(self.pointwise_conv3(x))

      x = F.relu(self.depthwise_conv4(x))
      x = F.relu(self.pointwise_conv4(x))

      x = F.relu(self.depthwise_conv5(x))
      x = F.relu(self.pointwise_conv5(x))

      x = F.relu(self.depthwise_conv6(x))
      x = F.relu(self.pointwise_conv6(x))

      for conv in self.five_conv:
            x = F.relu(conv(x))

      x = F.relu(self.depthwise_conv7(x))
      x = F.relu(self.pointwise_conv7(x))

      x = F.relu(self.depthwise_conv8(x))
      x = F.relu(self.pointwise_conv8(x))

      x = self.avg_pool(x)
      x = self.flatten(x)
      x = self.fc(x)

      return x

In [51]:
learning_rate = 0.0001
num_epochs = 10
batch_size = 32

In [52]:
model = NeuralNetwork_2().to(device)

In [53]:
from collections import Counter

# Training loop
def training():
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)

  train_labels = [label for _, label in train_dataset]
  class_counts = Counter(train_labels)
  total_samples = len(train_labels)
  class_weights = {cls: total_samples / count for cls, count in class_counts.items()}
  weight_tensor = torch.tensor([class_weights[i] for i in range(len(class_counts))], dtype=torch.float32)
  weight_tensor = weight_tensor.to(device)

  criterion = nn.CrossEntropyLoss(weight=weight_tensor)

  for epoch in range(num_epochs):
      for i, (images, labels) in enumerate(tqdm(train_loader)):
          images = images.to(device)
          labels = labels.to(device)

          # Forward pass
          outputs = model(images)
          loss = criterion(outputs, labels)

          # Backward and optimize
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          if (i+1) % 100 == 0:
              print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

  print("Training finished!")

  # Testing the model
  model.eval() # Set the model to evaluation mode
  with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f"Test accuracy of the model on the {total} test images: {correct / total}")

In [54]:
training()

 62%|██████▏   | 101/163 [00:09<00:05, 10.39it/s]

Epoch [1/10], Step [100/163], Loss: 4.6445


 62%|██████▏   | 101/163 [00:09<00:05, 10.48it/s]

Epoch [2/10], Step [100/163], Loss: 4.6170


 61%|██████▏   | 100/163 [00:09<00:06, 10.34it/s]

...

Epoch [9/10], Step [100/163], Loss: 4.6164


 63%|██████▎   | 102/163 [00:09<00:05, 10.59it/s]

Epoch [10/10], Step [100/163], Loss: 4.6096


100%|██████████| 163/163 [00:15<00:00, 10.36it/s]


Training finished!
Test accuracy of the model on the 1736 test images: 0.055299539170506916
